In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import requests
import json
import time
import random
from ast import literal_eval

In [2]:
# Get lapto's headers: https://httpbin.org/anything
headers =  {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7", 
    #"Accept-Encoding": "gzip, deflate, br, zstd", 
    "Accept-Language": "en-US,en;q=0.9,es;q=0.8", 
    "Dnt": "1", 
    #"Host": "httpbin.org", 
    "Referer": "https://www.scraperapi.com/", 
    "Sec-Ch-Ua": "\"Google Chrome\";v=\"125\", \"Chromium\";v=\"125\", \"Not.A/Brand\";v=\"24\"", 
    "Sec-Ch-Ua-Mobile": "?0", 
    "Sec-Ch-Ua-Platform": "\"Windows\"", 
    "Sec-Fetch-Dest": "document", 
    "Sec-Fetch-Mode": "navigate", 
    "Sec-Fetch-Site": "cross-site", 
    "Sec-Fetch-User": "?1", 
    "Upgrade-Insecure-Requests": "1", 
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36", 
    "X-Amzn-Trace-Id": "Root=1-665ec732-28d2172a4721ed726fc59ceb"
  }

def get_page(an):
    URL = f"https://register.epo.org/smartSearch?query={an}" #https://register.epo.org/smartSearch?query= #https://register.epo.org/application?number=EP
    page = requests.get(URL,headers=headers)
    return page.text

In [3]:
def extraer_tabla(soup):
    tablas = soup.find_all('table')
    return tablas

In [4]:
def extraer_valores(texto):
    # Extraer número de patente
    patent = re.findall(r'<span class="highlight">EP<span class="highlight">(\d+)</span></span>', texto)
    if len(patent) == 0:
        patent = ''
    else:
        patent = 'EP' + patent[0]

    # Extraer nombre de la patente
    patent_name = re.findall(r'</span></span> - (.*?)</a>', texto)
    if len(patent_name) == 0:
        patent_name = ''
    else:
        patent_name = patent_name[0].strip()  

    # Extraer Status
    status = re.findall("""Status</td><td class="t2" colspan="3">.*?<br/>""", texto)
    if len(status) == 0:
        status = ''
    else:
        status = status[0]
        status = status.replace("""Status</td><td class="t2" colspan="3">""","").replace('<br/>','').strip()

    #Most Recent Event date
    most_recent_event_date = re.findall(r'Most recent event.*?<td class="t2">(.*?)</td>', texto, re.DOTALL)
    if len(most_recent_event_date) == 0:
        most_recent_event_date = ''
    else:
        most_recent_event_date = most_recent_event_date[0].strip()

    #Most recent event
    most_recent_event = re.findall(r'Most recent event.*?<td class="t3">(.*?)</td>', texto, re.DOTALL)
    if len(most_recent_event) == 0:
        most_recent_event = ''
    else:
        most_recent_event = most_recent_event[0].replace('<br/>', '').replace('\r\n', '').replace('\t', '').replace('\xa0', ' ').strip()

    #Divisional Applications
    divisional_apps = re.findall(r'<td class="t2" colspan="3">(EP\d+\.\d+)\s*\/\s*<a.*?>(EP\d+)</a>', texto)
    divisional_dict = dict(divisional_apps)

    # Extraer inventores
    inventors = re.findall(r'<td class="t2" colspan="3">(\d+)\xa0/\r\n\t(.*?)<br/>\r\n\t(.*?)<br/>\r\n\t(.*?)\r\n\t  / (.*?)<br/>', texto)
    inventors_dict = []
    for inv in inventors:
        inventor_info = {
            'number': inv[0],
            'name': inv[1],
            'address': inv[2] + ', ' + inv[3],
            'country': inv[4]
        }
        inventors_dict.append(inventor_info)

    # Extraer publicaciones 
    publications = re.findall(r'<td class="th">Type:\r\n\s*</td>\s*<td class="t2" colspan="2">(.*?)</td>\s*<td class="th">No.:</td>\s*<td class="t2" colspan="2">(.*?)</td>\s*<td class="th">Date:</td>\s*<td class="t2" colspan="2">(.*?)</td>\s*<td class="th">Language:</td>\s*<td class="t2" colspan="2">(.*?)</td>', texto)
    publications_list = []
    for pub in publications:
        # Extraer el texto del enlace si existe, o usar el texto plano si no hay enlace
        type_text = re.search(r'>([^<]+)</a>$', pub[0])
        if type_text:
            pub_type = type_text.group(1).strip()
        else:
            pub_type = re.sub(r'\xa0', ' ', pub[0]).strip()
        publication_info = {
            'type': pub_type,
            'number': re.sub(r'<.*?>', '', pub[1]).strip(),
            'date': pub[2].strip(),
            'language': pub[3].strip()
        }
        publications_list.append(publication_info)
    
    # """
    # # Extraer Priority numbers
    # #TODO: Extraer los priority numbers
    # priority_section = re.search(r'<td class="th" rowspan="\d+">Priority number, date</td>(.*?)<td class="t2" colspan="3">\[', texto, re.DOTALL)
    # if not priority_section:
    #     return []
    # priority_data = priority_section.group(1)
    
    # priority_numbers = re.findall(r'<td class="t2">([A-Z]+\d+)</td><td class="t3" colspan="2">(.*?)\s*Original published format:\s*(.*?)</td>', priority_data)
    # priority_list = []
    # for prio in priority_numbers:
    #     priority_info = {
    #         'number': prio[0],
    #         'date': prio[1].strip(),
    #         'original_format': prio[2].strip()
    #     }
    #     priority_list.append(priority_info)

    # #Opponents
    # #TODO: Extraer los priority numbers
    # oponentes_actuales = re.findall(r'<td class="t2" colspan="2">(\d+)\s*([\d.]+)\s*([\d.]+)\s*ADMISSIBLE<br/>(.*?)<br/>(.*?)<br/>(.*?)\s*/\s*(.*?)<br/>Opponent\'s representative<br/>(.*?)<br/>(.*?)<br/>(.*?)<br/>(.*?)<br/>(.*?)\s*/\s*(.*?)<br/>', texto)
    # print(oponentes_actuales)
    # oponentes_actuales_list = []
    # for oponente in oponentes_actuales:
    #     oponente_info = {
    #         'number': oponente[0],
    #         'date1': oponente[1],
    #         'date2': oponente[2],
    #         'name': oponente[3],
    #         'address': f"{oponente[4]}, {oponente[5]}",
    #         'country': oponente[6],
    #         'representative': oponente[7],
    #         'rep_address': f"{oponente[8]}, {oponente[9]}, {oponente[10]}, {oponente[11]}",
    #         'rep_country': oponente[12]
    #     }
    #     oponentes_actuales_list.append(oponente_info)

    # # Extraer oponentes anteriores (former)
    # oponentes_former = re.findall(r'<tr class="former">.*?Former \[(.*?)\].*?<td class="t2" colspan="2">(\d+)\s*([\d.]+)\s*([\d.]+)\s*ADMISSIBLE<br/>(.*?)<br/>(.*?)<br/>(.*?)\s*/\s*(.*?)<br/>Opponent\'s representative<br/>(.*?)<br/>(.*?)<br/>(.*?)<br/>(.*?)<br/>(.*?)\s*/\s*(.*?)<br/>', texto, re.DOTALL)
    
    # oponentes_former_list = []
    # for oponente in oponentes_former:
    #     oponente_info = {
    #         'version': oponente[0],
    #         'number': oponente[1],
    #         'date1': oponente[2],
    #         'date2': oponente[3],
    #         'name': oponente[4],
    #         'address': f"{oponente[5]}, {oponente[6]}",
    #         'country': oponente[7],
    #         'representative': oponente[8],
    #         'rep_address': f"{oponente[9]}, {oponente[10]}, {oponente[11]}, {oponente[12]}",
    #         'rep_country': oponente[13]
    #     }
    #     oponentes_former_list.append(oponente_info)
    # """

    # Crear DataFrame
    df = pd.DataFrame({
        'Patent': [patent],
        'Patent_Name': [patent_name],
        'Status': [status],
        'Most_Recent_Event_Date': [most_recent_event_date],
        'Most_Recent_Event': [most_recent_event],
        'Divisional_applications': [json.dumps(divisional_dict)],
        'Inventors': [json.dumps(inventors_dict)],
        'Publications': [json.dumps(publications_list)]
    })
    
    return df


htmlContent = get_page(1125359)
if(htmlContent and "Just a moment..." not in htmlContent):
    soup = BeautifulSoup(htmlContent, 'html.parser')
    mytable = extraer_tabla(soup)[0]
    texto_limpio = extraer_valores(str(mytable))
    print(texto_limpio)
else:
    print("WebScrapping no disponible")

      Patent                                        Patent_Name  \
0  EP1125359  METHOD AND SYSTEM FOR DOWN-CONVERTING AN ELECT...   

                                  Status Most_Recent_Event_Date  \
0  No opposition filed within time limit             28.03.2008   

                                   Most_Recent_Event  \
0  Lapse of the patent in a contracting stateNew ...   

         Divisional_applications  \
0  {"EP02079190.1": "EP1315285"}   

                                           Inventors  \
0  [{"number": "01", "name": "SORRELLS, David, F....   

                                        Publications  
0  [{"type": "A1 Application with search report",...  


In [5]:
htmlContent = get_page(1999020)
if(htmlContent and "Just a moment..." not in htmlContent):
    soup = BeautifulSoup(htmlContent, 'html.parser')
    mytable = extraer_tabla(soup)[0]
    texto_limpio = extraer_valores(str(mytable))
    print(texto_limpio)
else:
    print("WebScrapping no disponible")

      Patent                                        Patent_Name  \
0  EP1999020  STRUCTURE FOR AN AIR INLET LIP FOR A NACELLE W...   

                                  Status Most_Recent_Event_Date  \
0  No opposition filed within time limit             15.07.2016   

                                   Most_Recent_Event Divisional_applications  \
0  Lapse of the patent in a contracting stateNew ...                      {}   

                                           Inventors  \
0  [{"number": "01", "name": "VAUCHEL, Guy, Berna...   

                                        Publications  
0  [{"type": "A1 Application with search report",...  


In [6]:
htmlContent = get_page(1492505)
if(htmlContent and "Just a moment..." not in htmlContent):
    soup = BeautifulSoup(htmlContent, 'html.parser')
    mytable = extraer_tabla(soup)[0]
    texto_limpio = extraer_valores(str(mytable))
    print(texto_limpio)
else:
    print("WebScrapping no disponible")


      Patent                                        Patent_Name  \
0  EP1492505  PHARMACEUTICAL PREPARATION CONTAINING OXYCODON...   

           Status Most_Recent_Event_Date  \
0  Patent revoked             26.05.2017   

                            Most_Recent_Event  \
0  Lapse of the patent in a contracting state   

                             Divisional_applications  \
0  {"EP10176720.0": "EP2319496", "EP11177513.6": ...   

                                           Inventors  \
0  [{"number": "01", "name": "BR\u00d6GMANN, Bian...   

                                        Publications  
0  [{"type": "A2 Application without search repor...  


In [7]:
json_str = texto_limpio['Divisional_applications'][0]

# Convertir la cadena JSON en un diccionario de Python
json_dict = json.loads(json_str)

# Obtener el primer key del diccionario
primer_key = list(json_dict.keys())[0]

primer_key

'EP10176720.0'

Acá comienzo a unir los df's para que ya se haga todo en conjunto

In [8]:
# df1 = pd.read_csv('/Users/diego/Documents/Universidades/Complutense/TFM/Sandov/DatosDivisionals20110101_20110630.csv')
# df2 = pd.read_csv('/Users/diego/Documents/Universidades/Complutense/TFM/Sandov/DatosDivisionals20110701_20111231.csv')
# df3 = pd.read_csv('/Users/diego/Documents/Universidades/Complutense/TFM/Sandov/DatosDivisionals20120101_20120630.csv')
# df4 = pd.read_csv('/Users/diego/Documents/Universidades/Complutense/TFM/Sandov/DatosDivisionals20120701_20121231.csv')
# df5 = pd.read_csv('/Users/diego/Documents/Universidades/Complutense/TFM/Sandov/DatosDivisionals20130101_20130631.csv')
# df6 = pd.read_csv('/Users/diego/Documents/Universidades/Complutense/TFM/Sandov/DatosDivisionals20130601_20131231.csv')
# df7 = pd.read_csv('/Users/diego/Documents/Universidades/Complutense/TFM/Sandov/DatosDivisionals20140101_20140631.csv')
# df8 = pd.read_csv('/Users/diego/Documents/Universidades/Complutense/TFM/Sandov/DatosDivisionals20140601_20141231.csv')
# dfAll = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], axis=0, ignore_index=True)``

In [9]:
df1 = pd.read_csv('/Users/diego/Documents/Universidades/Complutense/TFM/Sandov/Datos/Patents20090101_20090531.csv')
df2 = pd.read_csv('/Users/diego/Documents/Universidades/Complutense/TFM/Sandov/Datos/Patents20090531_20091231.csv')
dfAll = pd.concat([df1, df2], axis=0, ignore_index=True)

In [10]:
dfAll = dfAll.drop_duplicates()
dfAll

,Unnamed: 0,family,publication_number,application_number,ipc
0,0,89987512,P0700339,P0700339,A61P
1,1,35394157,P0700044,P0700044,A61P
2,2,89987139,P0600847,P0600847,A61P
3,3,89985663,P0402490,P0402490,A61P
4,4,26805159,09C0004,09C0004,A61P
...,...,...,...,...,...
19229,9900,35613661,88018,A200702125,A61K
19230,9901,34979083,88012,A200612978,A61K
19231,9902,35124997,88008,A200611517,A61K
19232,9903,32322658,88002,A200610767,A61K


In [11]:
dfs = []

for index, row in dfAll.iterrows():
    codigo = row['publication_number']
    print(codigo)
    htmlContent = get_page(codigo)
    soup = BeautifulSoup(htmlContent, 'html.parser')
    mytable = extraer_tabla(soup)[0]
    texto_limpio = extraer_valores(str(mytable))
    dfs.append(texto_limpio)
    time.sleep(15)#Para no sobrecargar la api

resultado_final = pd.concat(dfs, ignore_index=True)

P0700339


IndexError: list index out of range

In [ ]:
resultado_final

In [ ]:
resultado_final.to_csv("/Users/diego/Documents/Universidades/Complutense/TFM/Sandov/DatosDFFinal.csv")

In [ ]:
df_a_iterar = pd.read_csv('/Users/diego/Documents/Universidades/Complutense/TFM/Sandov/DatosDFFinal.csv')

In [ ]:
df_a_iterar

In [ ]:
df_a_iterar['Divisional_applications'] = df_a_iterar['Divisional_applications'].apply(literal_eval)

# Ahora, extraemos los valores de los diccionarios
nuevo_df = pd.DataFrame({    
    'publication_number': df_a_iterar['Divisional_applications'].apply(lambda x: list(x.values()))
})

# Expandimos la lista de valores en columnas separadas
nuevo_df = nuevo_df.explode('publication_number')

In [ ]:
nuevo_df = nuevo_df.dropna()
nuevo_df = nuevo_df.reset_index(drop=True)
nuevo_df

Hemos generado una nueva lista en base a los Publication number obtenidos del Scrappeo, volvemos a iterar sobre estos

In [ ]:
dfs = []

for index, row in nuevo_df.iterrows():
    codigo = row['publication_number']
    print(codigo)
    htmlContent = get_page(codigo)
    soup = BeautifulSoup(htmlContent, 'html.parser')
    mytable = extraer_tabla(soup)[0]
    texto_limpio = extraer_valores(str(mytable))
    dfs.append(texto_limpio)
    time.sleep(15)#Para no sobrecargar la api

resultado_final_segunda_iteracion = pd.concat(dfs, ignore_index=True)

In [ ]:
resultado_final_segunda_iteracion

In [ ]:
df_exterior = pd.DataFrame(columns=['publication_number'])

df_inicial = pd.read_csv('/Users/diego/Documents/Universidades/Complutense/TFM/Sandov/DatosDFFinal.csv')

def procesar_df(df):
    df['Divisional_applications'] = df['Divisional_applications'].apply(literal_eval)
    nuevo_df = pd.DataFrame({
        'publication_number': df['Divisional_applications'].apply(lambda x: list(x.values()))
    })
    nuevo_df = nuevo_df.explode('publication_number')
    nuevo_df = nuevo_df.dropna()
    nuevo_df = nuevo_df.reset_index(drop=True)
    return nuevo_df

df_actual = df_inicial

for iteracion in range(5):
    print(f"Iniciando iteración {iteracion + 1}")
    
    df_procesado = procesar_df(df_actual)
    
    df_exterior = pd.concat([df_exterior, df_procesado[['publication_number']]], ignore_index=True)
    
    dfs = []
    for index, row in df_procesado.iterrows():
        codigo = row['publication_number']
        print(f"Procesando código: {codigo}")
        htmlContent = get_page(codigo)
        soup = BeautifulSoup(htmlContent, 'html.parser')
        mytable = extraer_tabla(soup)[0]
        texto_limpio = extraer_valores(str(mytable))
        dfs.append(texto_limpio)
        time.sleep(15)  # Para no sobrecargar la api
    
    resultado_iteracion = pd.concat(dfs, ignore_index=True)
    
    if 'Divisional_applications' in resultado_iteracion.columns and not resultado_iteracion['Divisional_applications'].isna().all():
        df_actual = resultado_iteracion
    else:
        print(f"No hay más aplicaciones divisionales después de la iteración {iteracion + 1}")
        break

    print(f"Finalizada iteración {iteracion + 1}")

df_exterior